In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import *
import zipfile
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
local_zip = '/content/drive/My Drive/Dog-breed-identification/train.zip'
zip1 = zipfile.ZipFile(local_zip,'r')
zip1.extractall('/content/drive/My Drive/Dog-breed-identification/train')
zip1.close()

In [ ]:
local_zip = '/content/drive/My Drive/Dog-breed-identification/test.zip'
zip2 = zipfile.ZipFile(local_zip,'r')
zip2.extractall('/content/drive/My Drive/Dog-breed-identification/test')
zip2.close()

In [3]:
train_path = '/content/drive/My Drive/Dog-breed-identification/train/*'
test_path = '/content/drive/My Drive/Dog-breed-identification/test/*'
labels = '/content/drive/My Drive/Dog-breed-identification/labels.csv'
df = pd.read_csv(labels)
df.head()


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
breed = df['breed'].unique()
print(breed)
print("num of breed : ", len(breed))

['boston_bull' 'dingo' 'pekinese' 'bluetick' 'golden_retriever'
 'bedlington_terrier' 'borzoi' 'basenji' 'scottish_deerhound'
 'shetland_sheepdog' 'walker_hound' 'maltese_dog' 'norfolk_terrier'
 'african_hunting_dog' 'wire-haired_fox_terrier' 'redbone'
 'lakeland_terrier' 'boxer' 'doberman' 'otterhound' 'standard_schnauzer'
 'irish_water_spaniel' 'black-and-tan_coonhound' 'cairn' 'affenpinscher'
 'labrador_retriever' 'ibizan_hound' 'english_setter' 'weimaraner'
 'giant_schnauzer' 'groenendael' 'dhole' 'toy_poodle' 'border_terrier'
 'tibetan_terrier' 'norwegian_elkhound' 'shih-tzu' 'irish_terrier'
 'kuvasz' 'german_shepherd' 'greater_swiss_mountain_dog' 'basset'
 'australian_terrier' 'schipperke' 'rhodesian_ridgeback' 'irish_setter'
 'appenzeller' 'bloodhound' 'samoyed' 'miniature_schnauzer'
 'brittany_spaniel' 'kelpie' 'papillon' 'border_collie' 'entlebucher'
 'collie' 'malamute' 'welsh_springer_spaniel' 'chihuahua' 'saluki' 'pug'
 'malinois' 'komondor' 'airedale' 'leonberg' 'mexican_h

In [5]:
breed2id = {name : i for i, name in enumerate(breed)}
id2breed = {i : name for i, name in enumerate(breed)}
ids = glob(train_path)

labels = []

In [6]:
for image_id in ids:                 
     image_id = image_id.split("/")[-1].split(".")[0]                
     breed_name = list(df[df.id==image_id]["breed"])[0]
     breed_idx = breed2id[breed_name]               
     labels.append(breed_idx)

In [7]:
train_x, valid_x = train_test_split(ids, test_size = 0.25, random_state = 42)
train_y, valid_y = train_test_split(labels, test_size = 0.25, random_state = 42)
print(len(train_x))
print(len(train_y))

7666
7666


In [8]:
size = 224 
classes = 120
batch = 16
lr = 1e-5


In [9]:
def build_model(size, classes):
  inputs = Input((size, size, 3))
  backbone = MobileNetV2(input_tensor=inputs, include_top=False, weights='imagenet') 
  backbone.trainable = True
  x = backbone.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.5)(x)
  x = Dense(1024, kernel_regularizer=regularizers.l2(0.003), activation = 'relu')(x)
  x = Dense(1024, kernel_regularizer=regularizers.l2(0.003), activation = 'relu')(x)
  x = Dense(1024, kernel_regularizer=regularizers.l2(0.003), activation = 'relu')(x)
  x = Dense(classes, activation = 'softmax')(x)
  model = tf.keras.Model(inputs, x)
  return model

In [10]:
model = build_model(size, classes)
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr), metrics = ['accuracy'])
model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________

In [11]:
def read_image(path, size):     
     image = cv2.imread(path, cv2.IMREAD_COLOR)     
     image = cv2.resize(image, (size, size))     
     image = image / 255.0     
     image = image.astype(np.float32)     
     return image

def tf_parse(x, y):
    def _prase(x, y):
        x = x.decode()

        num_classes = 120
        size = 224
        image = read_image(x, size)
        label = [0] * num_classes
        label[y] = 1
        label = np.array(label, dtype=np.int32)
        return image, label

    x,y = tf.numpy_function(_prase, [x, y], [tf.float32, tf.int32])
    x.set_shape((224,224,3))
    y.set_shape((120))
    return x,y
def tf_dataset(x, y, batch=16):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    return dataset

In [12]:
train_dataset = tf_dataset(train_x, train_y)     
valid_dataset = tf_dataset(valid_x, valid_y) 

class myCallback(tf.keras.callbacks.Callback):
    def on_epoc_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.70 or logs.get('accuracy') > 0.95):
            print("Reached accuracy 70% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

train_steps = (len(train_x)//batch) + 1
valid_steps = (len(valid_x)//batch) + 1

In [14]:
model.fit(train_dataset, validation_data=valid_dataset, epochs=15, steps_per_epoch=train_steps, validation_steps=valid_steps, callbacks=[callbacks])

Epoch 1/15
480/480 [==============================] - 4576s 10s/step - loss: 14.1836 - accuracy: 0.0121 - val_loss: 13.8888 - val_accuracy: 0.0610
Epoch 2/15
480/480 [==============================] - 135s 282ms/step - loss: 13.6828 - accuracy: 0.0657 - val_loss: 13.3223 - val_accuracy: 0.1565
Epoch 3/15
480/480 [==============================] - 135s 281ms/step - loss: 13.0475 - accuracy: 0.1498 - val_loss: 12.4556 - val_accuracy: 0.2457
Epoch 4/15
480/480 [==============================] - 136s 284ms/step - loss: 12.1777 - accuracy: 0.2565 - val_loss: 11.4412 - val_accuracy: 0.3459
Epoch 5/15
480/480 [==============================] - 136s 283ms/step - loss: 11.3046 - accuracy: 0.3500 - val_loss: 10.6613 - val_accuracy: 0.4315
Epoch 6/15
480/480 [==============================] - 136s 283ms/step - loss: 10.5991 - accuracy: 0.4465 - val_loss: 10.1275 - val_accuracy: 0.4933
Epoch 7/15
480/480 [==============================] - 135s 282ms/step - loss: 10.0579 - accuracy: 0.5313 - val_lo

In [16]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = fn
  img = image.load_img(path, target_size = (224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 16)
  print(fn)
  
  breed_id = np.argmax(classes)
  name = id2breed[breed_id]
  print('the breed is: ',name)  

Saving lhasa.jpg to lhasa.jpg
lhasa.jpg
the breed is:  lhasa
